In [3]:
import psutil
def get_size(bytes, suffix="B"):
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor
print("="*40, "Memory Information", "="*40)
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")

======================================== Memory Information ========================================
Total: 25.51GB
Available: 24.54GB
Used: 631.64MB
Percentage: 3.8%


In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

work_path = '/content/gdrive/MyDrive/PaperRecommendation/'
data_path = work_path + "data/"
pickle_path = work_path + 'pickle/'
model_path = work_path + 'model_weight/'
checkpoint_path = work_path + "checkpoint/"

%run '/content/gdrive/MyDrive/PaperRecommendation/finalize_code_folder/function.ipynb'

Mounted at /content/gdrive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
3


In [5]:
class preprocess_text_04():
    def preprocess_Abstract(self, text):
        if type(text) == str:
            text = text.lower()

            # text = re.sub("[^\w]", " ", text)

            text = re.sub("[\d+]", "", text)

            text = re.sub("[\a-z]", " ", text)

            # text = re.sub("[^a-z| |A-Z]","", text)
            # text = re.sub(r"([a-z])([A-Z])", r"\1 \2", text)   # add " " between lowercase and uppercase : ArtificialIntelligence -> Artificial Intelligence

            # text = text.lower()
            # text = re.sub("ã|¢|î|º|â|å|½|¼|²|³", "", text)   # crawl error 

            filtered_sentence = [] 
            for w in text.split(" "): 
                if w not in stop_words and w not in stop:
                    filtered_sentence.append(w)
            text = " ".join(filtered_sentence)
            text = text.strip()
            text = re.sub("[ ]{2,}", " ", text)
        return text

    def preprocess_Title(self, text):
        return self.preprocess_Abstract(text)

    def preprocess_Aims(self, text):
        return self.preprocess_Abstract(text)
    
    def preprocess_Keywords(self, text):
        return self.preprocess_Abstract(text)

tool_preprocess = preprocess_tool(preprocess_text_04())
tool_NLP = NLP_tool()

In [6]:
# print("Preprocessing train ....")
# data_train = pd.read_csv(data_path + "train_ver2.csv", encoding = "ISO-8859-1")
# data_train = tool_preprocess.get_preprocessed_data(
#     data_train,
#     preprocess_columns = ['title', "abstract", "keywords"],
#     keep_columns = ['itr'],
#     new_column_names=['Title', 'Abstract', 'Keywords', "itr"],
#     n_jobs = 4
#     )

# print("Preprocessing validate ....")
# data_validate = pd.read_csv(data_path + "vali_ver2.csv", encoding = "ISO-8859-1")
# data_validate = tool_preprocess.get_preprocessed_data(
#     data_validate,
#     preprocess_columns = ['title', "abstract", "keywords"],
#     keep_columns = ['itr'],
#     new_column_names=['Title', 'Abstract', 'Keywords', "itr"],
#     n_jobs = 4
#     )

# print("Preprocessing test ....")
# data_test = pd.read_csv(data_path + "test_ver1.csv", encoding = "ISO-8859-1")
# data_test = tool_preprocess.get_preprocessed_data(
#     data_test,
#     preprocess_columns = ['title', "abstract", "keywords"],
#     keep_columns = ['itr'],
#     new_column_names=['Title', 'Abstract', 'Keywords', "itr"],
#     n_jobs = 4
#     )

# print("Preprocessing aims ....")
# data_aims = pd.read_csv(data_path + "aims_600.csv", encoding = "ISO-8859-1")
# data_aims = tool_preprocess.get_preprocessed_data(
#     data_aims,
#     preprocess_columns = ['aims'],
#     keep_columns = ['itr'],
#     new_column_names=['Aims', "itr"]
#     )

# data_train['Label'] = labelling_data(data_train['itr'], data_aims['itr'])
# data_validate['Label'] = labelling_data(data_validate['itr'], data_aims['itr'])
# data_test['Label'] = labelling_data(data_test['itr'], data_aims['itr'])


# data_train.to_csv(checkpoint_path + "preprocessed/02_train.csv", index=False)
# data_validate.to_csv(checkpoint_path + "preprocessed/02_validate.csv", index=False)
# data_test.to_csv(checkpoint_path + "preprocessed/02_test.csv", index=False)
# data_aims.to_csv(checkpoint_path + "preprocessed/02_aims.csv", index=False)

In [7]:
data_train = pd.read_csv(checkpoint_path + "preprocessed/02_train.csv", encoding = "ISO-8859-1")
data_validate = pd.read_csv(checkpoint_path + "preprocessed/02_validate.csv", encoding = "ISO-8859-1")
data_test = pd.read_csv(checkpoint_path + "preprocessed/02_test.csv", encoding = "ISO-8859-1")
data_aims = pd.read_csv(checkpoint_path + "preprocessed/02_aims.csv", encoding = "ISO-8859-1")

data_train.fillna("", inplace=True)
data_validate.fillna("", inplace=True)
data_test.fillna("", inplace=True)
data_aims.fillna("", inplace=True)

In [8]:
# corpus = (data_train['Title'] + ' ' + data_train['Abstract'] + " " + data_train['Keywords']).tolist()
# corpus = corpus + data_aims['Aims'].tolist()

# tokenizer = tool_NLP.build_tokenizer(corpus, vocabulary_size=None)
# with open(checkpoint_path + "pickle/02_tokenizer.pickle", 'wb') as f:
#     pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(checkpoint_path + "pickle/02_tokenizer.pickle", 'rb') as f:
    tokenizer = pickle.load(f)

if tokenizer.num_words == None:
    vocabulary_size = len(tokenizer.word_index)
else:
    vocabulary_size = tokenizer.num_words

print(vocabulary_size)

317033


In [9]:
# tool_NLP.download_FastText_pretrained()
# fasttext_embedding_dict = tool_NLP.build_fasttext_embedding_dict()
# embedding_matrix, zero_tokens = tool_NLP.build_embedding_matrix_FastText(tokenizer, fasttext_embedding_dict)


# with open(checkpoint_path + "pickle/02_embedding_matrix.pickle", 'wb') as f:
#     pickle.dump(embedding_matrix, f, protocol=pickle.HIGHEST_PROTOCOL)
# with open(checkpoint_path + "pickle/02_zero_tokens.pickle", "wb") as f:
#     pickle.dump(zero_tokens, f, protocol=pickle.HIGHEST_PROTOCOL)

with open(checkpoint_path + "pickle/02_embedding_matrix.pickle", 'rb') as f:
    embedding_matrix = pickle.load(f)
with open(checkpoint_path + "pickle/02_zero_tokens.pickle", 'rb') as f:
    zero_tokens = pickle.load(f)

display(zero_tokens)
# print(embedding_matrix.shape)

{1: 'â',
 114: 'î',
 268: 'î²',
 385: 'ã',
 679: 'pâ',
 759: 'ï',
 836: 'î¼m',
 1034: '²',
 1142: 'mgâ',
 1342: 'å',
 1411: 'nâ',
 1416: 'î¼g',
 1426: 'î³',
 1763: 'æ',
 2142: 'âµm',
 2199: 'câ',
 2276: 'ç',
 2300: 'solâ',
 2360: 'gâ',
 2486: 'uvâ',
 2784: 'îºb',
 2837: 'patientsâ',
 2854: 'ð',
 2906: 'alzheimerâ',
 2949: 'î¼',
 3044: 'orâ',
 3098: 'alâ',
 3155: 'aâ',
 3217: 'aî²',
 3327: 'îµ',
 3362: 'é',
 3477: 'niã',
 3479: 'childrenâ',
 3487: 'î¼gâ',
 3507: 'è',
 3638: 'covid',
 3800: 'âµg',
 3836: 'râ',
 3895: 'parkinsonâ',
 4110: 'theâ',
 4117: 'niâ',
 4243: 'cuâ',
 4254: 'ofâ',
 4256: 'ngâ',
 4266: 'mâ',
 4303: 'feâ',
 4308: 'oâ',
 4385: 'patientâ',
 4404: 'phâ',
 4642: 'womenâ',
 4758: 'á',
 4762: 'sâ',
 4812: 'ciâ',
 4888: 'hâ',
 4930: 'tâ',
 5014: 'lâ',
 5068: 'metalâ',
 5190: 'andâ',
 5209: 'studentsâ',
 5227: 'youngâ',
 5293: 'tiâ',
 5379: 'ä',
 5562: 'xâ',
 5729: 'coreâ',
 5740: 'caseâ',
 5758: 'hrâ',
 5811: 'iâ',
 5940: 'kgâ',
 5956: '¼',
 5967: 'dâ',
 6110: 'chromatograp

In [10]:
# tool_similarity = similarity_tool(embedding_matrix, zero_tokens)

# X_train = tool_NLP.tokenize_data(data_train['Title']
#                                  +' '+data_train['Abstract']
#                                  +' '+data_train['Keywords'],
#                                  tokenizer)
# centroid_train = tool_similarity.calculate_centroid_data(X_train)
# print(X_train.shape)

# X_validate = tool_NLP.tokenize_data(data_validate['Title']
#                                     +' '+data_validate['Abstract']
#                                     +' '+data_validate['Keywords'], 
#                                     tokenizer)
# centroid_validate = tool_similarity.calculate_centroid_data(X_validate)
# print(X_validate.shape)


# X_test = tool_NLP.tokenize_data(data_test['Title']
#                                 +' '+data_test['Abstract']
#                                 +' '+data_test['Keywords'],
#                                 tokenizer)
# centroid_test = tool_similarity.calculate_centroid_data(X_test)
# print(X_test.shape)

# tokenized_aims = tool_NLP.tokenize_data(data_aims['Aims'], tokenizer)
# centroid_aims = tool_similarity.calculate_centroid_data(tokenized_aims)
# print(tokenized_aims.shape)


# tool_similarity.init_cosine_matrix_aims(centroid_aim

# t1 = time.time()
# cosine_train = tool_similarity.calculate_cosine_similarity_with_aims(centroid_train)
# t2 = time.time()
# print("Calculate cosine_train runtime: %.f s "%(t2 - t1))
# with open(checkpoint_path + "pickle/02_cosine_train.pickle", 'wb') as f:
#     pickle.dump(cosine_train, f, protocol=pickle.HIGHEST_PROTOCOL)

# t1 = time.time()
# cosine_validate = tool_similarity.calculate_cosine_similarity_with_aims(centroid_validate)
# t2 = time.time()
# print("Calculate cosine_validate runtime: %.f s "%(t2 - t1))
# with open(checkpoint_path + "pickle/02_cosine_validate.pickle", 'wb') as f:
#     pickle.dump(cosine_validate, f, protocol=pickle.HIGHEST_PROTOCOL)

# t1 = time.time()
# cosine_test = tool_similarity.calculate_cosine_similarity_with_aims(centroid_test)
# t2 = time.time()
# print("Calculate cosine_test runtime: %.f s "%(t2 - t1))
# with open(checkpoint_path + "pickle/02_cosine_test.pickle", 'wb') as f:
#     pickle.dump(cosine_test, f, protocol=pickle.HIGHEST_PROTOCOL)


with open(checkpoint_path + "pickle/02_cosine_train.pickle", 'rb') as f:
    cosine_train = pickle.load(f)
with open(checkpoint_path + "pickle/02_cosine_validate.pickle", 'rb') as f:
    cosine_validate = pickle.load(f)
with open(checkpoint_path + "pickle/02_cosine_test.pickle", 'rb') as f:
    cosine_test = pickle.load(f)

print(cosine_train.shape)
print(cosine_validate.shape)
print(cosine_test.shape)

(298317, 351)
(33147, 351)
(83048, 351)


In [11]:
from sklearn.preprocessing import OneHotEncoder
num_labels = len(data_aims)

onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder.fit( np.array(range(num_labels)).reshape(-1,1) )
Y_train = onehot_encoder.transform(np.array(data_train['Label']).reshape((-1,1)))
Y_validate = onehot_encoder.transform(np.array(data_validate['Label']).reshape((-1,1)))
Y_test = onehot_encoder.transform(np.array(data_test['Label']).reshape((-1,1)))

In [12]:
import tensorflow as tf
import tensorflow.keras as keras
import functools     # special library for loss function in keras


from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dropout, Flatten, Concatenate, GlobalMaxPooling1D, Activation, GRU, MaxPooling1D
from tensorflow.keras.layers import Dense, Conv1D, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding

top3_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=3)
top3_acc.__name__ = "top3"

top5_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=5)
top5_acc.__name__ = "top5"

top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = "top10"

In [13]:
corpus_train = (
    data_train['Title']  
    + " " + data_train['Abstract']
    + " " + data_train['Keywords']
    )
corpus_validate = (
    data_validate['Title']  
    + " " + data_validate['Abstract']
    + " " + data_validate['Keywords']
    )
corpus_test = (
    data_test['Title']  
    + " " + data_test['Abstract']
    + " " + data_test['Keywords']
    )

X_train = tool_NLP.tokenize_data(corpus_train, tokenizer, maxlen = 700)
X_validate = tool_NLP.tokenize_data(corpus_validate, tokenizer, maxlen = 700)
X_test = tool_NLP.tokenize_data(corpus_test, tokenizer, maxlen = 700)

print(X_train.shape)
print(X_validate.shape)
print(X_test.shape)

(298317, 700)
(33147, 700)
(83048, 700)


# Run 1 without Aims

In [ ]:
shape_01 = X_train.shape[1]

inputs_01 = Input(shape = (shape_01,) )    # data tokenized max length
embedding_01 = Embedding(embedding_matrix.shape[0], 300, 
                         input_length = shape_01 ,
                         weights = [embedding_matrix],
                         trainable = False)(inputs_01)

conv1d_01 = Conv1D(filters=1800, kernel_size=2, activation='relu')(embedding_01)
pool_01 = GlobalMaxPooling1D()(conv1d_01)
flat_01 = Flatten()(pool_01)
dense_01 = Dense(1000, activation='relu')(flat_01)
drop_01 = Dropout(0.2)(dense_01)

                       
outputs = Dense(num_labels, activation='softmax')(drop_01)    # aims length is also number label
model = Model(inputs = inputs_01, outputs = outputs)

model.compile(loss='categorical_crossentropy', 
              optimizer="Adam",
              metrics=["accuracy",top3_acc,top5_acc,top10_acc])


print(model.summary())

mc = keras.callbacks.ModelCheckpoint(filepath = checkpoint_path + "weight/02_noAims_TAK.h5", 
                                     monitor='val_accuracy',
                                     mode='max', 
                                     verbose=1, 
                                     save_best_only=True)
# model = load_model('filename')
history = model.fit(X_train, Y_train,
                    batch_size=512,
                    epochs=30,
                    validation_data=(X_validate, Y_validate),
                    callbacks=[mc])

matrix_predict = model.predict(X_test)
Y_predict = np.argsort(matrix_predict, axis=1)[:,::-1]
GroundTruth = np.array(data_test['Label'])
for top_k in [1,3,5,10]:
    count = 0
    nPoints = len(GroundTruth)
    for i in range(nPoints):
        if GroundTruth[i] in Y_predict[i, 0:top_k]:
            count += 1
    print("top {} : {}".format(top_k,count/nPoints))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 700)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 700, 300)          95110200  
_________________________________________________________________
conv1d (Conv1D)              (None, 699, 1800)         1081800   
_________________________________________________________________
global_max_pooling1d (Global (None, 1800)              0         
_________________________________________________________________
flatten (Flatten)            (None, 1800)              0         
_________________________________________________________________
dense (Dense)                (None, 1000)              1801000   
_________________________________________________________________
dropout (Dropout)            (None, 1000)              0     

# Run 2 Aims & Scopes

In [14]:
shape_1 = X_train.shape[1]
shape_2 = cosine_train.shape[1]


inputs1 = Input(shape=(shape_1,) )    # data tokenized max length
inputs2 = Input(shape=(shape_2,))

#cosine
batch2_1 = tf.keras.layers.BatchNormalization()(inputs2)
dense2_1 = Dense(1500,activation="relu")(batch2_1)
drop2_1 = tf.keras.layers.Dropout(0.4)(dense2_1)
dense2_2 = Dense(1000,activation="relu")(drop2_1)
drop2_2 = tf.keras.layers.Dropout(0.4)(dense2_2)
dense2_3 = Dense(500,activation="relu")(drop2_2)
drop2_3 = tf.keras.layers.Dropout(0.4)(dense2_3)
#flow_main

embedding1 = Embedding(embedding_matrix.shape[0], 300, 
                      input_length = shape_1 ,
                      weights = [embedding_matrix],
                      trainable = False)(inputs1)

conv1 = Conv1D(filters=1800, kernel_size=2, activation='relu')(embedding1)
drop1_1 = Dropout(0.3)(conv1)
pool1_1 = GlobalMaxPooling1D()(drop1_1)

dense1_1 = Dense(1000, activation='relu')(pool1_1)
drop1_2 = Dropout(0.3)(dense1_1)
dense1_2 = Dense(500,activation="relu")(drop1_2)
drop1_3 = Dropout(0.3)(dense1_2)

concat = tf.keras.layers.Concatenate(axis=1)([drop2_3, drop1_3])

dense = Dense(500,activation="relu")(concat)
drop = Dropout(0.3)(dense)
                       
outputs = Dense(num_labels, activation='softmax')(drop)    # aims length is also number label
model = Model(inputs=[inputs1,inputs2], outputs=outputs)

model.compile(loss='categorical_crossentropy', 
              optimizer="Adam",
              metrics=["accuracy",top3_acc,top5_acc,top10_acc])

print(model.summary())

mc = keras.callbacks.ModelCheckpoint(filepath = checkpoint_path + "weight/02_Aims_TAK.h5", 
                                     monitor='val_accuracy',
                                     mode='max', 
                                     verbose=1, 
                                     save_best_only=True)

# model = load_model('filename')
history = model.fit([X_train, cosine_train], Y_train,
                    batch_size=512,
                    epochs=30,
                    validation_data=([X_validate, cosine_validate], Y_validate),
                    callbacks=[mc])


matrix_predict = model.predict([X_test, cosine_test])
Y_predict = np.argsort(matrix_predict, axis=1)[:,::-1]
GroundTruth = np.array(data_test['Label'])
for top_k in [1,3,5,10]:
    count = 0
    nPoints = len(GroundTruth)
    for i in range(nPoints):
        if GroundTruth[i] in Y_predict[i, 0:top_k]:
            count += 1
    print("top {} : {}".format(top_k,count/nPoints))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 700)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 351)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 700, 300)     95110200    input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 351)          1404        input_2[0][0]                    
______________________________________________________________________________________________